In [1]:
from bs4 import BeautifulSoup
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import psycopg2
import numpy as np

#### Setup database follow by https://github.com/mediagis/nominatim-docker/tree/master/4.0

In [2]:
conn = psycopg2.connect(database = "nominatim", user = "nominatim", password = "very_secure_password", host = "127.0.0.1", port = "5432")

In [3]:
cur = connect.cursor()
cur.execute(''' SELECT osm_id
                FROM placex
                WHERE name is not null and osm_type='W';''')
rows = cur.fetchall()
conn.commit()
conn.close()

In [5]:
osm_id_list = list()
for i in range (len(rows)):
    osm_id_list.append(rows[i][0])

In [6]:
len(osm_id_list)

156366

In [ ]:
url = "http://localhost:8080/lookup?osm_ids=W"
i = 1 

for osm_id in range(1,len(osm_id_list)) : 
    content=requests.get(url + str(osm_id_list[osm_id])).text
    xml_text = BeautifulSoup(content)

    df = pd.DataFrame()
    state = region = city = county = district = city_district = town = suburb = village = road = neighbourhood = ''
    if xml_text.find('state'):
        state = xml_text.state.text
    if xml_text.find('region'):
        region = xml_text.region.text
    if xml_text.find('city'):
        city = xml_text.city.text
    if xml_text.find('county'):
        county = xml_text.county.text
    if xml_text.find('district'):
        district = xml_text.district.text
    if xml_text.find('city_district'):
        city_district = xml_text.city_district.text
    if xml_text.find('town'):
        town = xml_text.town.text
    if xml_text.find('suburb'):
        suburb = xml_text.suburb.text
    if xml_text.find('village'):
        village = xml_text.village.text
    if xml_text.find('road'):
        road = xml_text.road.text
    if xml_text.find('neighbourhood'):
        neighbourhood = xml_text.neighbourhood.text
    try: 
        address = xml_text.place.attrs['display_name']
        osm_id = xml_text.place.attrs['osm_id']
    except:
        print(osm_id, xml_text)
        continue
    df = pd.DataFrame([[osm_id,state,region,city,county,district,city_district,town,suburb,village,road,neighbourhood,address]],
                      columns=['osm_id','state','region','city','county','district','city_district','town','suburb','village','road','neighbourhood','address'])
    
    if i == 1 :
        df.to_csv('./data/raw-osm-data.csv', mode='w', index=False, header=True)
        i = 2
    else:
        df.to_csv('./data/raw-osm-data.csv', mode='a', index=False, header=False)